# Concepts of Agentic AI

* agentic ai behaves with agency and autonomy, can problem-solve and react to different real-life situation
* flipped interaction pattern -> agentic system interacts with humans and other systems, without us necessary telling it what to do, in order to accomplish a goal we gave it

* agent loop (general)
    * human sets a goal
    * ai takes actions towards the goal using available resources (APIs, SQLs, etc)
        * prompt -> response -> action
        * get feedback on result from ai
    * termination criteria reached, return result

* agent loop
    * construct prompt
    * generate response
    * parse response
    * execute action
    * convert results to string
    * continue?

In [1]:
import os
with open("openai_token", "r") as file:
    openai_token = file.read().strip()
os.environ["OPENAI_API_KEY"] = openai_token

In [13]:
# scaffolding example
import base64
from typing import List, Dict
from litellm import completion

def generate_response(messages:List[Dict]) -> str:
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024,
        #temperature=0.7
    )
    return response.choices[0].message.content

messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming. Return Base64 encoded string only"},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

ZGVmIHN3YXBfZGljdChkKTogDQogICAgcmV0dXJuIHt2OiBrIGZvciBrLCB2IGluIGQuaXRlbXMoKX0=


In [15]:
# lets try a bit of programmatic prompting
import json

code_spec = {
    "name": "swap_keys_values",
    "description": "A function that swaps the keys and values in a dictionary.",
    "parameters": {
        "d": "A dictionary with unique values."}
    }

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"}
]

response = generate_response(messages)
print(response)

```python
def swap_keys_values(d):
    """
    Swaps the keys and values in a dictionary where all values are unique.
    
    Parameters:
    d (dict): A dictionary with unique values.
    
    Returns:
    dict: A new dictionary with keys and values swapped. The original values become keys,
          and the original keys become values.
    
    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}
    """
    return {value: key for key, value in d.items()}
```

This function creates a new dictionary by iterating over the items of the input dictionary and constructs key-value pairs by reversing the original key-value relationship. Since the original values are required to be unique, they can safely be used as keys in the new dictionary.


In [16]:
# models do not recall previous conversations, so they need to be supplied in the messaging part
messages = [
   {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
   {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."},
   
   # Here is the assistant's response from the previous step
   # with the code. This gives it "memory" of the previous
   # interaction.
   {"role": "assistant", "content": response},
   {"role": "user", "content": "Update the function to include documentation."}
]

response = generate_response(messages)
print(response)

Certainly! The function already contains a docstring for documentation, but let me expand it a little further to make it more comprehensive, detailing more about the assumptions and functionality:

```python
def swap_keys_values(d):
    """
    Swaps the keys and values in a dictionary, assuming all values are unique.
    
    Parameters:
    d (dict): A dictionary where all the values are unique and hashable. 
    
    Returns:
    dict: A new dictionary with the keys and values swapped. The values of the input
          dictionary become the keys, and the keys become the corresponding values.
          
    Raises:
    ValueError: If any duplicates are found among the values in the input dictionary,
                as they would cause collisions in the new dictionary where those values
                become keys.

    Example:
    >>> swap_keys_values({'a': 1, 'b': 2, 'c': 3})
    {1: 'a', 2: 'b', 3: 'c'}

    Note:
    - This function assumes that all values in the input dictionary

* system -> setting up the agent, most important part
* user -> user inputs
* assistant -> agent outputs  
**-> memory through user + assistant history**

In [51]:
import json
from typing import List, Dict
from litellm import completion


def generate_response(messages:List[Dict]) -> str:
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024,
        #temperature=0.7
    )
    return response.choices[0].message.content

In [58]:
def develop_custom_function():
    print("Enter function description:")
    user_input = input().strip()

    # first prompt
    messages = [
    {"role": "system",
        "content":
            """You are an expert software engineer that prefers functional
            programming.You return parsable json only, with explanations under
            key exp, code under key code, and func name under key name."""},
    {"role": "user",
        "content":
            user_input}]

    response = generate_response(messages)
    initial_func = json.loads(response)["code"]
    func_name = json.loads(response)["name"]
    print(initial_func)

    # second prompt, with memory of the first
    messages.append({"role": "assistant", "content": response})
    messages.append({"role": "user", "content":
            """Update the function to include clean documentation string,
            parameter descriptions, return value description,
            example usage, edge cases."""})

    response = generate_response(messages)
    documented_func = json.loads(response)["code"]
    print(documented_func)

    # third prompt, with memory of the first and second
    messages.append({"role": "assistant", "content": response})
    messages.append({"role": "user", "content":
        """Generate unit test for the function using unittest framework,
        test should cover typical usage, edge cases, error cases and various
        input types"""})

    response = generate_response(messages)
    test_cases = json.loads(response)["code"]
    print(test_cases)
    # 
    file_name = func_name+".py"
    with open(file_name, "w") as file:
        file.write(documented_func)
        file.write("\n\n")
        file.write(test_cases)

    return documented_func, test_cases, file_name

if __name__ == "__main__":
    func, tests, file_name = develop_custom_function()
    print(f"\nFunction and tests written to {file_name}")
    print("\nFunction:\n", func)
    print("\nTests:\n", tests)

Enter function description:
from sklearn.model_selection import RandomizedSearchCV


def random_hyperparam_optimization(model, param_distributions, X_train, y_train,
                                   cv=5, scoring='accuracy', n_iter=10, random_state=None, n_jobs=-1):
    """
    Perform random hyperparameter optimization for a given model.

    Parameters:
    - model: The machine learning model instance (e.g., RandomForestClassifier).
    - param_distributions: Dictionary with parameters names as keys and distributions or lists of parameters to try.
    - X_train: Training feature data.
    - y_train: Training labels.
    - cv: Number of cross-validation folds. Default is 5.
    - scoring: A single string to evaluate the predictions on the test set. Default is 'accuracy'.
    - n_iter: Number of parameter settings that are sampled. Default is 10.
    - random_state: Seed for random number generation. Default is None.
    - n_jobs: Number of jobs to run in parallel. Default is -1 (use

# Adding structure to AI Agents Outputs

* how to prompt a model to get an action that can be executed on behalf of the agent
* AI (LLMs) vs Environment interface (executing actions)
    * Prompt engineering + parsing
    * LLMs that support function calls


### Prompt engineering + parsing

* construct prompt
    * consistant format of the response
    * create template to follow (action:object)
    * LLMs can follow instructions (placeholders) nicely
* generate response
* parse response
    * rigorous, strict output format